## Importing libraries and loading dataset

In [1]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.nn import Embedding
from sklearn.model_selection import train_test_split
print("Modules imported")

Modules imported


In [2]:
df = pd.read_csv('train.csv', parse_dates=['date'], index_col = ['date'])
df.head()

,store,item,sales
date,,,
2013-01-01,1,1,13
2013-01-02,1,1,11
2013-01-03,1,1,14
2013-01-04,1,1,13
2013-01-05,1,1,10


In [3]:
test = df[df.index.year == 2017]
test.reset_index(level=0, inplace= True)
train = df[df.index.year != 2017]
train.reset_index(level = 0, inplace = True)
train.head()

,date,store,item,sales
0,2013-01-01,1,1,13
1,2013-01-02,1,1,11
2,2013-01-03,1,1,14
3,2013-01-04,1,1,13
4,2013-01-05,1,1,10


# Dataset pre-processing

In [4]:
train_data = pd.DataFrame({'year': train['date'].dt.year-2013, 'month': train['date'].dt.month,
                           'day': train['date'].dt.day, 'weekday': train['date'].dt.weekday,
                           'store': train['store'], 'item': train['item'], 'sales': train['sales']},
                          columns =['year', 'month', 'day', 'weekday', 'store', 'item', 'sales'])

test_data = pd.DataFrame({'year': test['date'].dt.year-2013, 'month': test['date'].dt.month,
                           'day': test['date'].dt.day, 'weekday': test['date'].dt.weekday,
                           'store': test['store'], 'item': test['item'], 'sales': test['sales']},
                          columns =['year', 'month', 'day', 'weekday', 'store', 'item', 'sales'])


In [5]:
print(train_data.head())
print(train_data.shape)

   year  month  day  weekday  store  item  sales
0     0      1    1        1      1     1     13
1     0      1    2        2      1     1     11
2     0      1    3        3      1     1     14
3     0      1    4        4      1     1     13
4     0      1    5        5      1     1     10
(730500, 7)


In [6]:
X = np.array(train_data.drop('sales', axis = 1))
y = np.array(train_data['sales'])
X_test = np.array(test_data.drop('sales', axis = 1))
y_test = np.array(test_data['sales'])

In [7]:

def split_data(X_train, y_train, val_ratio = 0.2, val_year = 3, half_yearly = 1, randomly = True):
    
    # Splitting randomly
    if randomly:
        X_tr, y_tr, X_val, y_val = train_test_split(X_train, y_train, test_size = (val_ratio),
                                                          random_state = 6, shuffle = True)
    else:
        if half_yearly == 1:        #if validation data is first 6 months of val_year
            
            X_tr = X_train[(X_train[:,0]!=val_year) | (X_train[:,1]>6)]   #if not val_year or in last 6 months of year
            y_tr = y_train[(X_train[:,0]!=val_year) | (X_train[:,1]>6)]
            
            X_val = X_train[(X_train[:,0]==val_year) & (X_train[:,1]<=6)] #if val_year and first 6 months of year
            y_val = y_train[(X_train[:,0]==val_year) & (X_train[:,1]<=6)]
            
        else:                       #if validation data is last 6 months of val_year
            
            X_tr = X_train[(X_train[:,0]!=val_year) | (X_train[:,1]<=6)]  #if not val_year or in first 6 months of year
            y_tr = y_train[(X_train[:,0]!=val_year) | (X_train[:,1]<=6)]
            
            X_val = X_train[(X_train[:,0]==val_year) & (X_train[:,1]>6)]  #if val_year and last 6 months of year
            y_val = y_train[(X_train[:,0]==val_year) & (X_train[:,1]>6)]
            
        return X_tr, y_tr, X_val, y_val

In [8]:
X_train, y_train, X_val, y_val = split_data(X, y, False, 0.3, 3, 0)
print("Training:", X_train.shape, y_train.shape)
print("Validation:", X_val.shape, y_val.shape)

Training: (730500, 6) (730500,)
Validation: (0, 6) (0,)


# Creating Long Short Term Memory model

In [9]:
# For embeddings, the thumb rule is, num_embeddings = no. of unique valus in category + 1 
# & embedding_dim = min(50,feat_dim(num_embeddings)/2)
dims = [np.unique(X_train[:,i]).size+1 for i in range(X_train.shape[1])]
print(dims)
embedding_dim = [(x, min(50, (x+1)//2)) for x in dims]
print(embedding_dim)

[5, 13, 32, 8, 11, 51]
[(5, 3), (13, 7), (32, 16), (8, 4), (11, 6), (51, 26)]


In [10]:
# Creating class for LSTM
class LSTMwithEmbeddings(nn.Module):
    def __init__(self, embedding_dim, n_cont, out_size, layers, dp = 0.3): #n_cont=no. of cont. feat. in dataframe
        super().__init__()
        self.embeds = nn.ModuleList([nn.Embedding(inp,out) for inp,out in embedding_dim])
        #print(self.embeds)
        self.emb_drop = nn.Dropout(dp)
        
        layer_list = []
        n_emb = sum((out for inp,out in embedding_dim))
        n_in = n_emb + n_cont
        n_hidden = [112, 96]
        n_dense = [64, 16]
        
        self.lstm1 = nn.LSTM(n_in, n_hidden[0], 1, batch_first = True) #(no. of inputs, hidden_size, num_layers)
        self.lstm2 = nn.LSTM(n_hidden[0], n_hidden[1], 1, batch_first = True)
        n_in_dense = n_hidden[1]    
        for i in n_dense:
            layer_list.append(nn.Linear(n_in_dense, i))
            layer_list.append(nn.ReLU(inplace = True))
            #layer_list.append(nn.BatchNorm1d(i))
            n_in_dense = i
            
        layer_list.append(nn.Dropout(dp))
        layer_list.append(nn.Linear(n_in_dense, out_size))

        self.dense_layers = nn.Sequential(*layer_list)


    def forward(self, X_cat, X_cont):
        embeddings = []
        n_hidden = [112, 96]
        batch_size = 365
        seq_len = 1
        for i, e in enumerate(self.embeds):
            embeddings.append(e(X_cat[:,i]))
        X = torch.cat(embeddings, axis =1)
        X = self.emb_drop(X)
        X = torch.cat([X, torch.unsqueeze(X_cont,1)], 1)
        
        h_1 = torch.randn(1, X.size(0), n_hidden[0]) #hidden state  ##(num_layers, batch_size, hidden_size)
        c_1 = torch.randn(1, X.size(0), n_hidden[0]) #internal state/cell state
        
        h_2 = torch.randn(1, X.size(0), n_hidden[1]) #hidden state
        c_2 = torch.randn(1, X.size(0), n_hidden[1]) #internal state/cell state
        
        h_out1, c_out1 = self.lstm1(X.reshape(X.size(0), 1, X.size(1)), (h_1,c_1)) #((batch_size(no. of elements in batch),seq_len, input_shape),(hidden))
        h_out2, c_out2 = self.lstm2(h_out1, (h_2,c_2))
        
        out = self.dense_layers(h_out2)
        return out.reshape(out.size(0),-1)


In [11]:
# Initiating LSTM model with 2 LSTM layers, 2 Fully connected layers and an output layer 
# having 112,96,64 & 16 nodes respectively

LSTMmodel = LSTMwithEmbeddings(embedding_dim[1:], 1, 1,2, 0)
LSTMmodel

LSTMwithEmbeddings(
  (embeds): ModuleList(
    (0): Embedding(13, 7)
    (1): Embedding(32, 16)
    (2): Embedding(8, 4)
    (3): Embedding(11, 6)
    (4): Embedding(51, 26)
  )
  (emb_drop): Dropout(p=0, inplace=False)
  (lstm1): LSTM(60, 112, batch_first=True)
  (lstm2): LSTM(112, 96, batch_first=True)
  (dense_layers): Sequential(
    (0): Linear(in_features=96, out_features=64, bias=True)
    (1): ReLU(inplace=True)
    (2): Linear(in_features=64, out_features=16, bias=True)
    (3): ReLU(inplace=True)
    (4): Dropout(p=0, inplace=False)
    (5): Linear(in_features=16, out_features=1, bias=True)
  )
)

In [12]:
# custom loss function (for optional use)
def smape(x,y):
    return 100*torch.mean(2*torch.abs(x-y)/(torch.abs(x)+torch.abs(y)))

optim = torch.optim.Adam(LSTMmodel.parameters(), lr = 0.01)
lossfn = F.mse_loss

# Training

In [13]:
def fit(epochs, sets, model, X_train, y_train, lossfn, optimizer):
    for i in range(sets//2):
        for j in range(2):
            X_tr, y_tr, X_val, y_val = split_data(X_train, y_train, val_year=i, half_yearly=j, randomly=False)
            losses = []
            for k in range(epochs):
                k+=1
                y_pred = LSTMmodel(torch.from_numpy(X_tr[:,1:]), torch.from_numpy(X_tr[:,0]))
                y_tr = torch.tensor(y_tr,dtype=torch.float).reshape(-1,1)
                loss = lossfn(y_pred,torch.tensor(y_tr))
                losses.append(loss)
                #if k%2 == 1:
                print("Epoch number {} of validation year 20{} and half {} has MSE loss {}".format(k,13+i,j,loss.item()))
                    
                loss.backward()
                optimizer.step()
                optimizer.zero_grad()

                if k%2 == 0:
                    with torch.no_grad():
                        yhat_val = LSTMmodel(torch.from_numpy(X_val[:,1:]), torch.from_numpy(X_val[:,0]))
                        y_val = torch.tensor(y_val,dtype=torch.float).reshape(-1,1)
                        val_loss = torch.sqrt(lossfn(torch.tensor(y_val), yhat_val))
                    print("Validation loss at epoch {} of year 20{} half {} is {:.4f}".format(k,13+i,j,val_loss.item()))


In [14]:
import time

In [15]:
begin = time.time()
fit(15, 8, LSTMmodel, X, y, lossfn, optim)
end = time.time()


/tmp/ipykernel_6182/3471745253.py:10: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  loss = lossfn(y_pred,torch.tensor(y_tr))


Epoch number 1 of validation year 2013 and half 0 has MSE loss 3464.2373046875


/tmp/ipykernel_6182/3471745253.py:9: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  y_tr = torch.tensor(y_tr,dtype=torch.float).reshape(-1,1)


Epoch number 2 of validation year 2013 and half 0 has MSE loss 3456.761474609375


/tmp/ipykernel_6182/3471745253.py:23: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  val_loss = torch.sqrt(lossfn(torch.tensor(y_val), yhat_val))


Validation loss at epoch 2 of year 2013 half 0 is 51.3380
Epoch number 3 of validation year 2013 and half 0 has MSE loss 3444.312255859375
Epoch number 4 of validation year 2013 and half 0 has MSE loss 3419.999267578125


/tmp/ipykernel_6182/3471745253.py:22: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  y_val = torch.tensor(y_val,dtype=torch.float).reshape(-1,1)


Validation loss at epoch 4 of year 2013 half 0 is 50.7352
Epoch number 5 of validation year 2013 and half 0 has MSE loss 3368.896240234375
Epoch number 6 of validation year 2013 and half 0 has MSE loss 3264.26953125
Validation loss at epoch 6 of year 2013 half 0 is 48.3136
Epoch number 7 of validation year 2013 and half 0 has MSE loss 3067.145263671875
Epoch number 8 of validation year 2013 and half 0 has MSE loss 2738.499755859375
Validation loss at epoch 8 of year 2013 half 0 is 41.0697
Epoch number 9 of validation year 2013 and half 0 has MSE loss 2262.511474609375
Epoch number 10 of validation year 2013 and half 0 has MSE loss 1677.7618408203125
Validation loss at epoch 10 of year 2013 half 0 is 27.8425
Epoch number 11 of validation year 2013 and half 0 has MSE loss 1113.8936767578125
Epoch number 12 of validation year 2013 and half 0 has MSE loss 809.1383056640625
Validation loss at epoch 12 of year 2013 half 0 is 30.5546
Epoch number 13 of validation year 2013 and half 0 has MSE 

Epoch number 5 of validation year 2015 and half 1 has MSE loss 93.20987701416016
Epoch number 6 of validation year 2015 and half 1 has MSE loss 91.51852416992188
Validation loss at epoch 6 of year 2015 half 1 is 9.2564
Epoch number 7 of validation year 2015 and half 1 has MSE loss 90.13162231445312
Epoch number 8 of validation year 2015 and half 1 has MSE loss 88.64959716796875
Validation loss at epoch 8 of year 2015 half 1 is 9.0840
Epoch number 9 of validation year 2015 and half 1 has MSE loss 87.43643951416016
Epoch number 10 of validation year 2015 and half 1 has MSE loss 85.87007141113281
Validation loss at epoch 10 of year 2015 half 1 is 8.9370
Epoch number 11 of validation year 2015 and half 1 has MSE loss 84.58817291259766
Epoch number 12 of validation year 2015 and half 1 has MSE loss 83.570068359375
Validation loss at epoch 12 of year 2015 half 1 is 8.8025
Epoch number 13 of validation year 2015 and half 1 has MSE loss 82.35394287109375
Epoch number 14 of validation year 2015

In [16]:
print("Time taken for training in a Ryzen 5 Hexa core 4600H CPU is {:.2f} minutes".format((end-begin)/60))

Time taken for training in a Ryzen 5 Hexa core 4600H CPU is 42.62 minutes


# Testing

In [17]:
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error, mean_absolute_percentage_error
#import torch_metrics as tm

In [18]:
def show_metrics(test, y_pred_final):
    metrics = {'R2_score': r2_score(test, y_pred_final), 'MAE': mean_absolute_error(test, y_pred_final),
               'RMSE': mean_squared_error(test, y_pred_final, squared=False),
               'MAPE': mean_absolute_percentage_error(test, y_pred_final)}
    adj_R2 = 1-(1-metrics['R2_score'])*(len(test)-1)/(len(test)-6-1)      #num of indep var = 6
    metrics['adj_R2'] = adj_R2
    print("R2 score on test set is", metrics['R2_score'])
    print("Mean Absolute Error on test set is", metrics['MAE'])
    print("Root Mean Square error on test set is", metrics['RMSE'])
    print("Mean Absolute Percentage Error on test set is", metrics['MAPE'])
    print("Adjusted R2 score on test set is", adj_R2)
    
    return metrics

In [19]:
predictions = LSTMmodel(torch.from_numpy(X_test[:,1:]), torch.from_numpy(X_test[:,0]))
test_results = show_metrics(y_test, predictions.detach().numpy())

R2 score on test set is 0.9130831700643351
Mean Absolute Error on test set is 7.067161381278626
Root Mean Square error on test set is 9.302233612700073
Mean Absolute Percentage Error on test set is 0.15067687205231767
Adjusted R2 score on test set is 0.9130803124151123


# Saving the model

In [20]:
torch.save(LSTMmodel.state_dict(), 'LSTM.pth')